<a href="https://colab.research.google.com/github/emredeveloper/Transformers--General-AI/blob/main/Projeksiyon_Katmanlar%C4%B1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import torch.nn as nn

# Model parametreleri
d_model = 64   # Modelin gizli boyutu
d_ff = 256     # Besleme ileri (feed-forward) boyutu
seq_len = 10   # Girdi sırasının uzunluğu
batch_size = 8 # Batch boyutu

In [3]:
# Girdi tensorü (örnek veriler)
input_tensor = torch.rand(batch_size, seq_len, d_model)  # Rastgele veri

In [4]:
# Projeksiyon Katmanları
class ProjectionLayers(nn.Module):
    def __init__(self, d_model, d_ff):
        super().__init__()
        # Query, Key, Value Projections
        self.q_proj = nn.Linear(d_model, d_model)
        self.k_proj = nn.Linear(d_model, d_model)
        self.v_proj = nn.Linear(d_model, d_model)
        self.o_proj = nn.Linear(d_model, d_model)

        # Feed-forward Projections
        self.gate_proj = nn.Linear(d_model, d_model)
        self.up_proj = nn.Linear(d_model, d_ff)
        self.down_proj = nn.Linear(d_ff, d_model)

    def forward(self, x):
        # Attention Projections
        q = self.q_proj(x)  # Sorgu
        k = self.k_proj(x)  # Anahtar
        v = self.v_proj(x)  # Değer

        # Dot-product attention için hazırlık
        attention_scores = torch.matmul(q, k.transpose(-2, -1)) / (d_model ** 0.5)
        attention_weights = torch.softmax(attention_scores, dim=-1)
        attention_output = torch.matmul(attention_weights, v)

        # Output Projeksiyon
        output = self.o_proj(attention_output)

        # Feed-forward katmanı
        gated = torch.sigmoid(self.gate_proj(output)) * output  # Gated mekanizma
        upsampled = self.up_proj(gated)  # Daha yüksek boyut
        downsampled = self.down_proj(upsampled)  # Boyut küçültme
        return downsampled

In [5]:
# Modeli tanımla ve çalıştır
model = ProjectionLayers(d_model=d_model, d_ff=d_ff)
output_tensor = model(input_tensor)

print("Girdi Boyutu: ", input_tensor.shape)
print("Çıktı Boyutu: ", output_tensor.shape)


Girdi Boyutu:  torch.Size([8, 10, 64])
Çıktı Boyutu:  torch.Size([8, 10, 64])
